# Fully Connected Neural Networks in TensorFlow's Keras


We will use dense neural networks in Keras to solve a simple regression problem. 

The dataset is a collection of simulated particle events from [Pythia](http://home.thep.lu.se/~torbjorn/Pythia.html). Specifically, we will construct a dense neural network which will learn how to calculate the invariant mass of a particle from its energy and momentum.

First, import `numpy`, `tensorflow`, and `pylab` and load the data.


In [ ]:
import numpy as np
import tensorflow as tf
import pylab as plt

In [ ]:
# load the data, which is stored as a numpy array data type
energy = np.load("../data/homogenous-16-particle-events-energy.npy")
print(energy.shape)

The dataset is a 2D array where each row represents one event of data from an electron-proton collision. This dataset is comprised _only_ of events where there were exactly 16 particles produced from an electron-proton collision. Each particle has an x-momentum, y-momentum, z-momentum, energy, and charge: $(p_x,p_y,p_z,E,q)$. Each event is therefore represented by 80 numbers. Using numpy's reshape method we can make each row represent one particle. 

In [ ]:
# Here we rearrange the data within each of the events to isolate particles
energy = np.reshape(energy, (len(energy), 16, 5))

# Use another call of reshape to combine all events
# Complete me:
# Target: (#events*16, 5)


These are our training data inputs, but we also must provide the targets, which are the invariant masses of each particle. This is a straightforward computation that does not require neural networks, but provides an easy-to-understand problem.

We choose units where $c = 1$:
$$m^2=E^2-||\textbf{p}||^2$$
where $m, E$, and $\textbf{p}$ are all in GeV.

In [ ]:
# Complete me:


There are several hundred thousand datapoints in this dataset which is overkill for this simple example. Create a test dataset with just 50 examples. (We're deliberately overcompensating here and making the dataset unreasonably small, to highlight some learning failure modes. Later in this notebook, we'll use all of the data.)

In [ ]:
#Slicing allows you select a subset of an array.
#This can be done like this: smallerArray = largerArray[:1000]

# Complete me:

Next, make a histogram of the target data to make sure that we are seeing masses of real particles. As this data has limited precision, this will not resolve electrons very well, but protons, pions, and massless particles should be clearly visible.

In [ ]:
plt.hist(target,bins=100)
plt.show()

Now we can build and train the first neural network. Start with a simple one hidden layer network with 5 neurons, and ReLU activation. Train for 30 epochs with a batch size of 50, an Adam optimizer with a learning rate of 0.1, using mean squared error loss. 

Use a standard validation split of 0.2 (i.e., 80% of the data will be used for training, the other 20% for validation).

Information on how to implement these features can be found here:
<https://www.tensorflow.org/api_docs/python/tf/keras>.
Check out Sequential underneath models and Dense under layers.


In [ ]:
model = tf.keras.Sequential() #Define the model object
model.add(tf.keras.layers.Dense(5, input_shape=(5,), activation="relu")) #Add the hidden layer

#Add the output layer yourself
#It should not have an activation function
#Complete me:


model.compile(tf.keras.optimizers.Adam(lr=0.1),loss=tf.keras.losses.MeanSquaredError()) #Adam optimizer and mean squared error loss
results = model.fit(smallEnergy, smallTarget, epochs=30, batch_size=50, validation_split=0.2)

In [ ]:
def plot_learning_curve(history):
    """ Function that accepts the result from a training run and generates loss curves. """
    plt.plot(history["loss"], label="training loss")
    plt.plot(history["val_loss"], label="validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend()
    plt.show()

plot_learning_curve(results.history)

The model is learning, but we can do better. Perhaps we did not have enough model parameters to accurately represent the mapping. Remedy this by increasing the number of hidden neurons to 20.

In [ ]:
#Use the same code as in the previous cells
#Simply change the number of neurons in the hidden layer

We see that we got little improvement here. Another hyperparameter to adjust is *batch size*, which is the number of training examples used to calculate the gradient on each step. While you may initially think that a higher batch size leads to faster or more accurate training, in practice this is not true. The "noise" that arises from using less training examples at each iteration can actually help find the global minimum of the loss function.
(See here for more info: https://arxiv.org/pdf/1609.04836.pdf)

Try decreasing the batch size to 16.

In [ ]:
#Complete me:


This is starting to do better but there is significant room for improvement.

Another hyperparameter to tune is the *learning rate*. 

 - If the learning rate is too high, we are taking too large of a step in the gradient descent at each iteration and may have difficulty converging to a minimium. 
 - If the learning rate is too small, then we are not traveling far enough in each iteration and we will take far too long to reach a minimum. 

Perhaps the learning rate is too high and the network can't fine tune. Try decreasing the learning rate to 0.001. With such a reduction in learning rate, you will typically need to also train for more longer to attain convergence --- you may want to double the number of epochs of training to 60 (or possibly more).

In [ ]:
#Reuse the code from the previous cells and change the appropriate hyperparameters
#Complete me:


While the learning curves have now gotten smoother, you may have noticed that the performance (particularly on the held out validation set) is not that much better. Indeed, there is now evidence of *overtraining* or *overfitting* -- the training loss is so much lower than the validation loss. 

One common fix to this is adding *dropout layers*, that serve as a form of regularization. Try adding a dropout layer with a dropout rate of 0.05. <https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout>

You can also try batch normalization: <https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization>

In [ ]:
#Dropout layers are located under tf.keras.layers. 
#They take the dropout rate as their only argument.
#BatchNormalization layers are also under tf.keras.layers, and in the simplest use case, take no arguments

#Complete me:



While this has somewhat closed the gap between the two loss curves, the performance is still unsatisfactory. Let's now try training on the full dataset. Use a single hidden layer with 20 neurons, a learning rate of 0.001, and a batch size of 256. You can eliminate the Dropout layer, since using more data already has a regularizing effect. Train it for just 10 epochs to keep the runtime shorter.

In [ ]:
#Complete me:


This clearly resulted in a significant improvement and shows how important having a large enough dataset is. Let's see if we can do even better by increasing *model capacity*, i.e., by building a neural network with more nodes, and thus, training a model with more parameters. Add 2 more hidden layers to the network, while fixing all other hyperparameters. How does the performance compare to the shallower model?

In [ ]:
#Complete me:


Clearly, adding more layers helps improve the quality of the network. But there is a limit to how effective this is though. What happens with 5 hidden layers?

In [ ]:
#Complete me:


Alternately, we can also experiment with the "width" of each hidden layer. What happens when you increase the number of neurons per layer to 50 in the 3 hidden layer model? Compare the performance to the previous model with 3 layers that used only 20 nodes in each layer. 

In [ ]:
#Complete me:


Let's now experiment with other activation functions. Try the sigmoid and the tanh activation functions, and in particular, see how well they perform with deep architectures (when compared to ReLU). Build a network with between 1--8 hidden layers with 50 nodes each. How does network performance vary as you change the number of hidden layers? Pay particular attention to the rate of convergence of the loss.

In [ ]:
#Simply replace "relu" with "sigmoid" or "tanh" to try other activation functions
#Complete me:


This difference in performance, especially when the sigmoid function is used in deeper networks, is caused by what's known as the vanishing gradient problem. If the value for any one the neurons gets too far away from 0, the gradient for sigmoid and tanh gets really close to 0. This means that for deeper networks it is much more difficult to update the weights in the earlier layers as their gradient is so small. Now, remove the fifth column from the input data, the charge, and see what happens when training (you may want to revert to the three-layer ReLU model for this). Why do you think including charge has such a large impact?

In [ ]:
#Complete me:


Finally, there are other options for the loss function. Try experimenting with alternatives to mean squared error.

<https://www.tensorflow.org/api_docs/python/tf/keras/losses>

You can also try some other optimizers -- for example, sgd (with and without momentum), rmsprop, adagrad, adadelta, adamax, and nadam. <https://www.tensorflow.org/api_docs/python/tf/keras/optimizers>



In [ ]:
#Complete me:
